In [1]:
import numpy as np

#
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.path as mpath
from matplotlib import colors
from matplotlib import cm
from matplotlib.axes import Axes
from cartopy.mpl.geoaxes import GeoAxes
GeoAxes._pcolormesh_patched = Axes.pcolormesh
import matplotlib
matplotlib.rcParams['agg.path.chunksize'] = 10000

from scipy.interpolate import interpn
import math
import warnings
%matplotlib inline
import xarray as xr
#import bisect

# import DBSCAN
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')

# on the header:
import matplotlib as mpl

In [2]:
import gsw

In [3]:
ds = xr.open_dataset('/scratch/project_2000789/muramarg/waom_yearly_floats/output_WAOM_check/ocean_flt.nc')
ds1 = xr.open_dataset('/scratch/project_2000789/muramarg/waom_yearly_floats/output_WAOM_check/ocean_avg_0001.nc')
dg = xr.open_dataset('/scratch/project_2000789/boeiradi/waom10_frc/waom10extend_grd.nc')

In [4]:
salt = ds.variables['salt'].values
temp = ds.variables['temp'].values
depth = ds.variables['depth'].values

In [5]:
salt.shape

(65717, 5460)

In [6]:
salt_flat = salt.flatten()
temp_flat = temp.flatten()
depth_flat = depth.flatten()
depth_flat = np.nan_to_num(depth_flat)

In [7]:
#x = x[~numpy.isnan(x)]
#salt_flat = salt_flat[~np.isnan(salt_flat)]
#depth_flat = depth_flat[~np.isnan(depth_flat)]
#depth_flat = depth_flat[~np.isnan(depth_flat)]

In [8]:
salt_flat = salt_flat[salt_flat != 0]
temp_flat = temp_flat[temp_flat != 0]

In [9]:
# # make grid for density contours
# ax.set_xlim([34,34.7])
# # ax.set_ylim([-2.7,-1])

smin = 31 - (0.01 * 31)    #salt_ctrl_subregR.min - (0.01 * salt_ctrl_subregR.min)
smax = 35. + (0.01 * 35.)    #salt_ctrl_subregR.max + (0.01 * salt_ctrl_subregR.max)
tmin = -3 + (0.1 * -3)       #temp_ctrl_subregR.min - (0.1 * temp_ctrl_subregR.max)
tmax = 3.3 + (0.1 * 3.3)       #temp_ctrl_subregR.max + (0.1 * temp_ctrl_subregR.max)
print('tmin, tmax, smin, smax sizes=,', tmin, tmax, smin, smax)
# Calculate how many gridcells we need in the x and y dimensions
xdim = 20
ydim = 20
# Create empty grid of zeros
dens = np.zeros((ydim,xdim))
# Create temp and salt vectors of appropiate dimensions
ti = np.linspace(-3,3.3,ydim)
si = np.linspace(31,35,xdim)

Si, Ti = np.meshgrid(si, ti, sparse=False, indexing='ij')
# Loop to fill in grid with densities
for j in range(0,int(ydim)):
    for i in range(0, int(xdim)):
        #print(si[i],ti[j])
        dens[j,i]=gsw.rho(si[i],ti[j],0)
        # Substract 1000 to convert to sigma-0
dens = dens - 1000
#print(np.max(dens),np.min(dens))

tmin, tmax, smin, smax sizes=, -3.3 3.63 30.69 35.35


### create the arrays for neutral density

In [10]:
# dens_neut = np.array([[ 24.71689606,  24.91883278,  25.12090874,  25.32289696,  25.52485657,
#    25.72688103 , 25.92889404 , 26.13092232 , 26.33299828 , 26.53503227,
#    26.73715973 , 26.93921471  ,27.14241028  ,27.34601402  ,27.54989433,
#    27.75070953  ,27.94353294 , 28.63241196,  28.98308754, 28.98308754    ],
# [ 24.71689606,  24.91883278,  25.12090874,  25.32289696,  25.52485657,
#    25.72688103 , 25.92889404 , 26.13092232 , 26.33299828 , 26.53503227,
#    26.73715973 , 26.93921471  ,27.14241028  ,27.34601402  ,27.54989433,
#    27.75070953  ,27.94353294 , 28.63241196,  28.98308754,  28.98308754      ],
#  [ 24.71689606,  24.91883278,  25.12090874,  25.32289696,  25.52485657,
#    25.72688103 , 25.92889404 , 26.13092232 , 26.33299828 , 26.53503227,
#    26.73715973 , 26.93921471  ,27.14241028  ,27.34601402  ,27.54989433,
#    27.75070953  ,27.94353294 , 28.63241196,  28.98308754,28.98308754  ],
#  [ 24.71313286 , 24.91477394 , 25.11647034 , 25.31818581  ,25.51997185,
#    25.72172546 , 25.92341423 , 26.12518883,  26.32695389 , 26.52869797,
#    26.73050308 , 26.93229294 , 27.13504791 , 27.33821678 , 27.54164696,
#    27.7422142  , 27.93387222 , 28.5552597  , 28.91858482 ,28.91858482],
#  [ 24.70738411 , 24.90876579 , 25.11021423 , 25.31170082 , 25.51305199,
#    25.71452713 , 25.91602516 , 26.11743355 , 26.31898117 , 26.52041817,
#    26.72193336 , 26.92341232 , 27.12577629 , 27.32852173 , 27.53152275,
#    27.73191071 , 27.92254639,  28.4789753  , 28.8517704  , 29.14905739],
#  [ 24.69975662 , 24.90088463 , 25.10199547 , 25.30318642 , 25.50432968,
#    25.70547485,  25.90666008 , 26.10786438 , 26.30905342 , 26.51026917,
#    26.71149445,  26.9126873  , 27.1146183  , 27.3169632 ,  27.51953316,
#    27.71981621 , 27.90965462  ,28.40401077 , 28.77214432  ,29.0795536 ],
#  [ 24.69028091 , 24.89105415,  25.09194946 , 25.2928524  , 25.49374962,
#    25.6946125 ,  25.89553261 , 26.09637833 , 26.29730034 , 26.49825478,
#    26.69918442 , 26.90014458 , 27.10159683 , 27.30356216 , 27.50571442,
#    27.70590782 , 27.89554596 , 28.31707764 , 28.69371796 , 29.00740814],
#  [ 24.6788826  , 24.87939453 , 25.07999802,  25.28060722 , 25.48117638,
#    25.68186188 , 25.88243866 , 26.08312035 , 26.28370094 , 26.48443604,
#    26.68505287 , 26.88575363 , 27.08674812 , 27.2883358  , 27.49008369,
#    27.69023705 , 27.87995338 , 28.22002792 , 28.61616516 , 28.93346214],
#  [ 24.66562653 , 24.86592484 , 25.06619453 , 25.26650429 , 25.4668808,
#    25.66721725 , 25.86752701 , 26.06794167 , 26.26830673 , 26.46874237,
#    26.66911507 , 26.86951637 , 27.07017708 , 27.27131462 , 27.47266197,
#    27.67278481,  27.86296654,  28.092659  ,  28.53930855 , 28.85771751],
#  [ 24.65052795 , 24.85054398 , 25.05057335 , 25.25064468 , 25.45073128,
#    25.650774   , 25.85088348 , 26.05101013 , 26.25109482 , 26.451231,
#    26.65131187 , 26.85151482 , 27.05180931 , 27.25252151 , 27.45347404,
#    27.6534996  , 27.84414101 , 28.05034637 , 28.46330833  ,28.7783432 ],
#  [ 24.63355637 , 24.83339119 , 25.03311157 , 25.23295021 , 25.43276787,
#    25.6325531   ,25.83239174 , 26.03218651 , 26.2320385  , 26.43192673,
#    26.6317749  , 26.83172035 , 27.03166008 , 27.23199844 , 27.43253708,
#    27.63247681 , 27.8237114  , 28.01834106 , 28.38858414 , 28.69993019],
#  [ 24.61491394  ,24.81440163 , 25.01390266 , 25.21342468 , 25.41304207,
#    25.61253357  ,25.81213379 , 26.01170158 , 26.21125984 , 26.41089249,
#    26.61052895  ,26.81011772 , 27.00978851 , 27.20977402 , 27.40987968,
#    27.60972595 , 27.80182266 , 27.98947525 , 28.3213501  , 28.62238312],
#  [ 24.59440041 , 24.79360771 , 24.99295044  ,25.19223595 , 25.39147949,
#    25.59079361  ,25.7900753  , 25.98940468 , 26.18877029 , 26.38808823,
#    26.58743286 , 26.78684998 , 26.98615265 , 27.18586349 , 27.38550377,
#    27.58516312 , 27.77878189 , 27.96234894 , 28.26257133 , 28.54547119],
#  [ 24.57218361 , 24.7711277  , 24.97018242 , 25.16918755 , 25.36826134,
#    25.56728363 , 25.76635742 , 25.96536827 , 26.16444778 , 26.36356735,
#    26.56266975 , 26.76180458  ,26.96086311 , 27.16026306 , 27.35944557,
#    27.55879211 , 27.7533474 ,  27.93611526 , 28.20168877 , 28.46933365],
#  [ 24.5481987 ,  24.74694443 , 24.94569016,  25.14442062 , 25.34326935,
#    25.54204369 , 25.74079895 , 25.9396553 ,  26.13848686 , 26.33733177,
#    26.53620338 , 26.73507118  ,26.93390083 , 27.13297081 , 27.33171082,
#    27.53074837 , 27.72612572  ,27.90915489 , 28.13986015 , 28.39437103],
#  [ 24.52243805 , 24.72093964 , 24.91951561 , 25.1179657  , 25.31651878,
#    25.51508522 , 25.71369553 , 25.91222572 , 26.11079597 , 26.30937958,
#    26.50796318 , 26.70662689,  26.90526772 , 27.10399818 , 27.30233574,
#    27.5010643  , 27.69707489 , 27.88111115 , 28.08379745 , 28.33390808],
#  [ 24.49506569 , 24.69329834 , 24.89158249 , 25.08985901 , 25.28813934,
#    25.48644066 , 25.68478012 , 25.88305855 , 26.08143044 , 26.27975845,
#    26.47814751 , 26.67650795 , 26.87494278 , 27.0733223  , 27.27133751,
#    27.4697628  , 27.66625595 , 27.85202599 , 28.04242706 , 28.28282928],
#  [ 24.46591759 , 24.66394806 , 24.86195183 , 25.05997849 , 25.25805283,
#    25.45612907  ,25.65417862 , 25.85229301 , 26.05039787,  26.24848557,
#    26.44663048 , 26.64472771 , 26.84292603 , 27.04101944 , 27.2387886,
#    27.43680763 , 27.63369942 , 27.82089424 , 28.00405693 , 28.23462677],
#  [ 24.43512344,  24.63295555 , 24.83068848  ,25.02856064 , 25.22626686,
#    25.42415428  ,25.62194633,  25.81985092 , 26.01771355 , 26.21559525,
#    26.41346169  ,26.61138916 , 26.80927467 , 27.00706482 , 27.20471191,
#    27.4022541  , 27.59948921 , 27.78889084 , 27.96739769,  28.18323135],
#  [ 24.40274429 , 24.60030937 , 24.79779625 , 24.99540329 , 25.19295502,
#    25.39055443 , 25.58813667 , 25.78577805 , 25.98334312 , 26.18100548,
#    26.37866402 , 26.5763855  , 26.77403831 , 26.97160721 , 27.16907501,
#    27.36613083 , 27.5632534  , 27.7547226 ,  27.93291092 , 28.13616371]])

In [11]:
ti

array([-3.        , -2.66842105, -2.33684211, -2.00526316, -1.67368421,
       -1.34210526, -1.01052632, -0.67894737, -0.34736842, -0.01578947,
        0.31578947,  0.64736842,  0.97894737,  1.31052632,  1.64210526,
        1.97368421,  2.30526316,  2.63684211,  2.96842105,  3.3       ])

## Define limits of salinity and temperature

In [12]:
salt_limit = np.arange(34.25,34.75,0.25)
temp_limit = np.arange(-1.7,-2,0.1)

## For each index released at release times, is it within the TS limits

In [13]:
x = (ds.variables['Xgrid'].values)

In [14]:
# Gade line: WDW (Weddell Deep Water)
T_wdw = -2
S_wdw = 34.5
Lf = 334 # kJ/kg
Cp = gsw.cp_t_exact(S_wdw,T_wdw,0)/1000
print(Cp)

Tgade = T_wdw + (Lf/Cp)*(1 - (S_wdw/si))

3.990697896299028


## Plotting all points with polygons

In [15]:
# # make grid for density contours
# ax.set_xlim([34,34.7])
# # ax.set_ylim([-2.7,-1])

smin = 33 - (0.01 * 33)    #salt_ctrl_subregR.min - (0.01 * salt_ctrl_subregR.min)
smax = 35. + (0.01 * 35.)    #salt_ctrl_subregR.max + (0.01 * salt_ctrl_subregR.max)
tmin = -3 + (0.1 * -3)       #temp_ctrl_subregR.min - (0.1 * temp_ctrl_subregR.max)
tmax = 3.3 + (0.1 * 3.3)       #temp_ctrl_subregR.max + (0.1 * temp_ctrl_subregR.max)
print('tmin, tmax, smin, smax sizes=,', tmin, tmax, smin, smax)
# Calculate how many gridcells we need in the x and y dimensions
xdim = 20
ydim = 20
# Create empty grid of zeros
dens = np.zeros((ydim,xdim))
# Create temp and salt vectors of appropiate dimensions
ti = np.linspace(-3,3.3,ydim)
si = np.linspace(33,35,xdim)

Si, Ti = np.meshgrid(si, ti, sparse=False, indexing='ij')
# Loop to fill in grid with densities
for j in range(0,int(ydim)):
    for i in range(0, int(xdim)):
        #print(si[i],ti[j])
        dens[j,i]=gsw.rho(si[i],ti[j],0)
        # Substract 1000 to convert to sigma-0
dens = dens - 1000
#print(np.max(dens),np.min(dens))

tmin, tmax, smin, smax sizes=, -3.3 3.63 32.67 35.35


In [16]:
depth = ds.variables['depth'].values
rho = ds.variables['rho'].values

In [17]:
# the mask will be based on rho, salt, and temp

# use the values from portella

In [18]:
allpts = np.arange(0,rho.shape[1],1)
allpts

array([   0,    1,    2, ..., 5457, 5458, 5459])

In [19]:
# for each index in each month, find the first values of T and S
hssw_idxs = np.array([],dtype=np.int64)
for idx in allpts:
    thisrow_T = temp[:,int(idx)]
    thisrow_salt = salt[:,int(idx)]
    thisrow_depth = depth[:,int(idx)]
    
    # remove nan values
    thisrow_T = thisrow_T[~np.isnan(thisrow_T)]
    thisrow_salt = thisrow_salt[~np.isnan(thisrow_salt)]
    thisrow_depth = thisrow_depth[~np.isnan(thisrow_depth)]
    
    # test if the first value is within TS limits
    if (len(thisrow_T)>0):
        t1 = thisrow_T[0]
        s1 = thisrow_salt[0]
        if (t1 <= -1.7 and t1 >= -2) and (s1 >= 34.5):
            # add it to the list of indices of interest
            hssw_idxs = np.append(hssw_idxs,int(idx))
print(len(hssw_idxs))

1445


In [20]:
all_salt = ds.variables['salt'].values[:,hssw_idxs]
all_temp = ds.variables['temp'].values[:,hssw_idxs]

In [21]:
all_depth = ds.variables['depth'].values[:,hssw_idxs]

In [22]:
Freez_temp = gsw.CT_freezing(si,0,0)

# Get an array of last T and S values of the hssw_idxs

In [23]:
final_s = all_salt[-1]
final_t = all_temp[-1]

In [29]:
with open('/scratch/project_2000789/muramarg/statistics/pygamman_f2py-master/pygamman_f2py-master/TS_nums.txt', 'w') as f:
    for i,j in zip(final_s,final_t):
        f.write(str(i)+"   "+str(j)+"   \n")
f.close()

In [24]:
with open("/scratch/project_2000789/muramarg/statistics/pygamman_f2py-master/pygamman_f2py-master/neutral_densities.txt","r") as g:
    gamma_n = g.read().splitlines() 
    gamma_n = [float(x) for x in gamma_n]

# Looping method of collecting indexes

#### Identify AABW points based on the definition in Descriptive Oceanography

In [25]:
len(gamma_n)
#len(hssw_idxs)

1445

In [26]:
Freez_temp

array([-1.79085645, -1.79694006, -1.80302639, -1.80911543, -1.81520719,
       -1.82130168, -1.8273989 , -1.83349884, -1.83960152, -1.84570694,
       -1.8518151 , -1.857926  , -1.86403964, -1.87015604, -1.87627519,
       -1.88239709, -1.88852176, -1.89464919, -1.90077938, -1.90691234])

In [27]:
(gamma_n[0])

27.897488

In [30]:
# color the points only if their endpoint is within AASW
# salinity < 34.5 and temperature greater than freezing line
# for each index in the previous figure
aabw_idx = np.array([],dtype=np.int64)
gi = 0
for idx in hssw_idxs:
    # get the final point in the row
    thisrow_T = temp[:,int(idx)]
    thisrow_salt = salt[:,int(idx)]
    thisrow_depth = depth[:,int(idx)]
    if gamma_n[gi]>28.27:
        # convert meters to ft
        #print(thisrow_depth[-1])
        if thisrow_depth[-1] < (-3500/3.281):
            #print(thisrow_salt[-1])
            if thisrow_salt[-1]>34.4 and thisrow_salt[-1]<34.75:
                # search sorted to find the closest salt value index
                #test_idx = np.searchsorted(si,thisrow_salt[-1])
                if thisrow_T[-1]<0:
                    # it is within AASW
                    aabw_idx = np.append(aabw_idx,int(idx))
    gi += 1
            
aabw_salt = ds.variables['salt'].values[:,aabw_idx]
aabw_temp = ds.variables['temp'].values[:,aabw_idx]
aabw_cmap = ds.variables['depth'].values[:,aabw_idx]
# for i in range(len(aasw_cmap)):
#      aasw_cmap[i][:]=i
print(len(aabw_idx))

349


In [31]:
aabw_cmap = ds.variables['depth'].values[:,aabw_idx]

In [32]:
aabw_cmap*=-1

In [33]:
# get first values in the array
firstsalts = np.array([],dtype=np.int64)
for i in range(len(aabw_salt[0])):
    thiscol = aabw_salt[:,i]
    firstval = next(x for x in thiscol if not np.isnan(x))
    firstsalts = np.append(firstsalts,firstval)
firsttemps = np.array([],dtype=np.int64)
for i in range(len(aabw_temp[0])):
    thiscol = aabw_temp[:,i]
    firstval = next(x for x in thiscol if not np.isnan(x))
    firsttemps = np.append(firsttemps,firstval)
    
# get the last values in the array
lastsalts = aabw_salt[-1]
lasttemps = aabw_temp[-1]

In [32]:
# plot floats ending as AASW colored by time
figure = plt.figure(figsize=[25, 25])
ax = plt.subplot(1,1,1)
plt.title('AABW')
plt.rcParams['axes.labelsize']='x-large'
plt.scatter(aabw_salt,aabw_temp,s=0.1,marker='o',c=aabw_cmap,cmap='viridis')
plt.scatter(firstsalts,firsttemps,marker='X',color='g',label="Starting points")
plt.scatter(lastsalts,lasttemps,marker='X',color='r',label="Endpoints")
#cbar = plt.colorbar(ticks=[0, 2820, 5640, 8460, 11280, 14100, 16920, 19740, 22560])
cbar = plt.colorbar()
#cbar.ax.set_yticklabels(['0','30','60','90','120','150','180','210','240'])
cbar.set_label('Depth', rotation=270,labelpad=15)
plt.plot(si,Freez_temp,'-',color='dimgray',label='Seawater Freezing',alpha=0.5,zorder=1)
#plt.plot(si,Tgade,'-',color='steelblue',label='Gade Line',alpha=0.5,zorder=2)
plt.ylim([-2.4,0.3])
plt.xlim([33.8,35.1])
plt.legend(loc="best",fontsize="x-large")
cs = ax.contour(si, ti, dens, colors="silver", zorder=1,alpha=0.5)
#ax.contour(si,ti,Freez_temp, colors="grey", zorder=1,alpha=0.5)
cl=plt.clabel(cs,fontsize=10,inline=False,fmt="%.1f")

plt.savefig("aabw_points.png", dpi=300)
plt.close()


KeyboardInterrupt



Error in callback <function flush_figures at 0x7f1831b61700> (for post_execute):


KeyboardInterrupt: 

### Group similar trajectories together

In [138]:
# flatten the data to create two x and y lines
x_flat = aabw_salt.flatten()
y_flat = aabw_temp.flatten()

# drop the NaN values from the list
x_flat = x_flat[~np.isnan(x_flat)]
y_flat = y_flat[~np.isnan(y_flat)]

In [139]:
aabw_salt.shape

(65717, 349)

In [140]:
x_flat

array([34.501606, 34.50235 , 34.50169 , ..., 34.64647 , 34.64194 ,
       34.645283], dtype=float32)

In [141]:
"""
    Fit the Data 
"""
X = [i for i in zip(x_flat,y_flat)]
X = StandardScaler().fit_transform(X)
X.shape

(16117444, 2)

In [38]:
X = X[:1000]

In [39]:
x = x_flat[:1000]
y = y_flat[:1000]

In [40]:
"""
    Compute the DBSCAN
"""
db = DBSCAN(eps=0.01, min_samples=1).fit(X)
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
#n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_clusters_ = 5
n_clusters_

5

In [41]:
print(set(labels))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30}


In [42]:
"""
    Plot the clusters
"""
d= dict(zip(set(labels),['red','green','blue','yellow']))
d[-1] = "black"
plt.scatter(x,y,color=[ d[i] for i in labels])
plt.show()

KeyError: 4

In [142]:
# try creating a number based on the variation and sum of the values in each row of aabw_salt and aabw_temp
# 3: points that go below the freezing line
# 1: points that go straight up (salinity does not become fresh)
# 2: points that become fresh then go back to AABW
# 0: points that do not fit in these categories

# first create an array of zeros to store the color values of the points
colors = np.zeros(aabw_salt.shape[1])
avg_freez = sum(Freez_temp)/len(Freez_temp)

# look at the variances to see what is normal
var = np.array([])

for i in range(aabw_salt.shape[1]):
    pt_salt = aabw_salt[:,i]
    pt_temp = aabw_temp[:,i]
    pt_salt = pt_salt[~np.isnan(pt_salt)]
    pt_temp = pt_temp[~np.isnan(pt_temp)]
    var = np.append(var,(np.var(pt_salt)))
    
    # does it go below the ice shelf
    if (min(pt_temp))<=avg_freez-0.2:
        colors[i] = 3
    # does the salinity not change
    elif (np.var(pt_salt)<0.003):
        colors[i] = 1
    # does it become AASW then go back to AABW
    elif (min(pt_salt))<34.4:
        colors[i] = 2

In [45]:
d= dict(zip(set(colors),['red','green','blue','yellow']))

In [46]:
# reshape the color array
colors1 = np.tile(colors, (aabw_salt.shape[0], 1))
colors1.shape

(65717, 349)

In [47]:
# plot floats ending as AASW colored by time
figure = plt.figure(figsize=[25, 25])
ax = plt.subplot(1,1,1)
plt.title('AABW')
plt.rcParams['axes.labelsize']='x-large'
plt.scatter(aabw_salt,aabw_temp,s=0.1,marker='o',c=colors1)
plt.scatter(firstsalts,firsttemps,marker='X',color='g',label="Starting points")
plt.scatter(lastsalts,lasttemps,marker='X',color='r',label="Endpoints")
#cbar = plt.colorbar(ticks=[0, 2820, 5640, 8460, 11280, 14100, 16920, 19740, 22560])
#cbar = plt.colorbar()
#cbar.ax.set_yticklabels(['0','30','60','90','120','150','180','210','240'])
#cbar.set_label('Depth', rotation=270,labelpad=15)
plt.plot(si,Freez_temp,'-',color='dimgray',label='Seawater Freezing',alpha=0.5,zorder=1)
#plt.plot(si,Tgade,'-',color='steelblue',label='Gade Line',alpha=0.5,zorder=2)
plt.ylim([-2.4,0.3])
plt.xlim([33.8,35.1])
plt.xlabel("Salinity",fontsize=15)
plt.ylabel("Temperature",fontsize=15)
plt.legend(loc="best",fontsize=30)
cs = ax.contour(si, ti, dens, colors="silver", zorder=1,alpha=0.5)
#ax.contour(si,ti,Freez_temp, colors="grey", zorder=1,alpha=0.5)
cl=plt.clabel(cs,fontsize=10,inline=False,fmt="%.1f")

plt.savefig("colored_pts.png", dpi=300)
plt.close()

### Find the mean of the labelled lines and plot them

In [48]:
# get indices in colors of interest
idx_0 = np.array([],dtype= np.int32)
idx_1 = np.array([],dtype= np.int32)
idx_2 = np.array([],dtype= np.int32)
idx_3 = np.array([],dtype= np.int32)

for i in range(len(colors)):
    if int(colors[i]) == 0:
        idx_0 = np.append(idx_0,i)
    elif int(colors[i]) == 1:
        idx_1 = np.append(idx_1,i)
    elif int(colors[i]) == 2:
        idx_2 = np.append(idx_2,i)
    elif int(colors[i]) == 3:
        idx_3 = np.append(idx_3,i)

In [49]:
salt_0 = aabw_salt[:,idx_0]
salt_1 = aabw_salt[:,idx_1]
salt_2 = aabw_salt[:,idx_2]
salt_3 = aabw_salt[:,idx_3]

temp_0 = aabw_temp[:,idx_0]
temp_1 = aabw_temp[:,idx_1]
temp_2 = aabw_temp[:,idx_2]
temp_3 = aabw_temp[:,idx_3]

In [50]:
# get the averages for each row
as_0 = np.mean(salt_0,axis=1)
as_1 = np.mean(salt_1,axis=1)
as_2 = np.mean(salt_2,axis=1)
as_3 = np.mean(salt_3,axis=1)

at_0 = np.mean(temp_0,axis=1)
at_1 = np.mean(temp_1,axis=1)
at_2 = np.mean(temp_2,axis=1)
at_3 = np.mean(temp_3,axis=1)

In [ ]:
# plot the average lines in a density plot

In [53]:
# plot floats ending as AASW colored by time
figure = plt.figure(figsize=[25, 25])
ax = plt.subplot(1,1,1)
plt.title('AABW')
plt.rcParams['axes.labelsize']='x-large'
plt.scatter(as_0,at_0,s=0.1,marker='o',c="red")
plt.scatter(as_1,at_1,s=0.1,marker='o',c="blue")
plt.scatter(as_2,at_2,s=0.1,marker='o',c="green")
plt.scatter(as_3,at_3,s=0.1,marker='o',c="yellow")
plt.plot(si,Freez_temp,'-',color='dimgray',label='Seawater Freezing',alpha=0.5,zorder=1)
#plt.plot(si,Tgade,'-',color='steelblue',label='Gade Line',alpha=0.5,zorder=2)
plt.ylim([-2.4,0.3])
plt.xlim([33.8,35.1])
plt.xlabel("Salinity",fontsize=15)
plt.ylabel("Temperature",fontsize=15)
plt.legend(loc="best",fontsize=30)
cs = ax.contour(si, ti, dens, colors="silver", zorder=1,alpha=0.5)
#ax.contour(si,ti,Freez_temp, colors="grey", zorder=1,alpha=0.5)
cl=plt.clabel(cs,fontsize=10,inline=False,fmt="%.1f")

plt.savefig("average_pts.png", dpi=300)
plt.close()

### Group trajectories with DBSCAN

In [108]:
# get an array of the variability and average salinity
# try creating a number based on the variation and sum of the values in each row of aabw_salt and aabw_temp
# 3: points that go below the freezing line
# 1: points that go straight up (salinity does not become fresh)
# 2: points that become fresh then go back to AABW
# 0: points that do not fit in these categories

# first create an array of zeros to store the color values of the points
varbs = np.zeros(aabw_salt.shape[1])
avg_sal = np.zeros(aabw_salt.shape[1])

for i in range(aabw_salt.shape[1]):
    pt_salt = aabw_salt[:,i]
    pt_temp = aabw_temp[:,i]
    pt_salt = pt_salt[~np.isnan(pt_salt)]
    pt_temp = pt_temp[~np.isnan(pt_temp)]
    
    # create an array of variability in salt
    #thisvar = np.var(pt_temp)
    #varbs[i]=(np.var(pt_salt) + thisvar)
    varbs[i]=(np.var(pt_salt))
    # create an array of averages in salt
    avg_sal[i] = (sum(pt_salt)/len(pt_salt))

In [109]:
X = [i for i in zip(varbs,avg_sal)]
X = StandardScaler().fit_transform(X)
X.shape

(349, 2)

In [110]:
"""
    Compute the DBSCAN
"""
db = DBSCAN(eps=0.08, min_samples=1).fit(X)
labels = db.labels_
print(np.bincount(labels))

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

# renumber labels as clusters 1-5
for i in range(len(labels)):
    if labels[i]<=20:
        labels[i] = 1
    elif labels[i]<=40:
        labels[i] = 2
    elif labels[i]<=60:
        labels[i] = 3
    elif labels[i]<=80:
        labels[i] = 4
    else:
        labels[i] = 5

[  1  35 145   2  17   6   1   1   6   5   1   1   1   2   1   2   2   1
   1   2   1   3   3   3   1   2   1  16   1   1   1   1   1   1   1   2
   1   1   1   1   1   1   1   1   2   1   1   1   1   1   1   2   1   1
   2   2   1   1   1   2   1   1   1   1   1   1   1   1   1   3   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1]


In [111]:
labels

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1,
       2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 2, 3, 3, 1, 2, 1, 1, 3, 1, 1,
       2, 2, 1, 3, 3, 3, 3, 3, 1, 2, 1, 1, 1, 1, 1, 3, 1, 3, 1, 3, 1, 3,
       2, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 3, 1, 3, 3, 1, 1, 3, 1, 3, 3, 1,
       1, 1, 1, 2, 1, 3, 2, 1, 1, 1, 1, 3, 1, 4, 4, 1, 1, 4, 4, 2, 4, 1,
       1, 1, 4, 1, 4, 2, 4, 1, 1, 1, 1, 1, 1, 2, 1, 1, 4, 1, 1, 1, 4, 4,
       1, 1, 4, 1, 1, 4, 4, 4, 2, 4, 1, 1, 4, 1, 4, 1, 4, 4, 1, 3, 5, 1,
       1, 1, 1, 1, 1, 1, 1, 5, 1, 5, 5, 5, 5, 5, 1, 3, 2, 1, 3, 1, 5, 5,
       1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 5, 1, 5, 5, 1, 1, 1, 4, 1, 1, 1,
       1, 1, 5, 1, 1, 1, 2, 1, 1, 1, 1, 5, 1, 1, 1,

In [112]:
# set the labels as the colors in the plot
colors1 = np.tile(labels, (aabw_salt.shape[0], 1))
colors1.shape

(65717, 349)

In [113]:
# plot floats ending as AASW colored by time
figure = plt.figure(figsize=[25, 25])
ax = plt.subplot(1,1,1)
plt.title('AABW')
plt.rcParams['axes.labelsize']='x-large'
plt.scatter(aabw_salt,aabw_temp,s=0.1,marker='o',c=colors1)
plt.scatter(firstsalts,firsttemps,marker='X',color='g',label="Starting points")
plt.scatter(lastsalts,lasttemps,marker='X',color='r',label="Endpoints")
plt.plot(si,Freez_temp,'-',color='dimgray',label='Seawater Freezing',alpha=0.5,zorder=1)
#plt.plot(si,Tgade,'-',color='steelblue',label='Gade Line',alpha=0.5,zorder=2)
plt.ylim([-2.4,0.3])
plt.xlim([33.8,35.1])
plt.xlabel("Salinity",fontsize=15)
plt.ylabel("Temperature",fontsize=15)
plt.legend(loc="best",fontsize=30)
cs = ax.contour(si, ti, dens, colors="silver", zorder=1,alpha=0.5)
#ax.contour(si,ti,Freez_temp, colors="grey", zorder=1,alpha=0.5)
cl=plt.clabel(cs,fontsize=10,inline=False,fmt="%.1f")

plt.savefig("dbscan_pts5_2vars.png", dpi=300)
plt.close()

In [134]:
# find the average of each color and plot these 5 trajectories
# get indices in colors of interest
idx_1 = np.array([],dtype= np.int32)
idx_2 = np.array([],dtype= np.int32)
idx_3 = np.array([],dtype= np.int32)
idx_4 = np.array([],dtype= np.int32)
idx_5 = np.array([],dtype= np.int32)
for i in range(len(labels)):
    if int(colors[i]) == 1:
        idx_1 = np.append(idx_1,i)
    elif int(colors[i]) == 2:
        idx_2 = np.append(idx_2,i)
    elif int(colors[i]) == 3:
        idx_3 = np.append(idx_3,i)
    elif int(colors[i]) == 4:
        idx_4 = np.append(idx_4,i)
    elif int(colors[i]) == 5:
        idx_5 = np.append(idx_5,i)

# 
salt_1 = aabw_salt[:,idx_1]
salt_2 = aabw_salt[:,idx_2]
salt_3 = aabw_salt[:,idx_3]
salt_4 = aabw_salt[:,idx_4]
salt_5 = aabw_salt[:,idx_5]
temp_1 = aabw_temp[:,idx_1]
temp_2 = aabw_temp[:,idx_2]
temp_3 = aabw_temp[:,idx_3]
temp_4 = aabw_temp[:,idx_4]
temp_5 = aabw_temp[:,idx_5]

# get the averages for each row
as_1 = np.nanmean(salt_1,axis=1)
as_2 = np.nanmean(salt_2,axis=1)
as_3 = np.nanmean(salt_3,axis=1)
as_4 = np.nanmean(salt_4,axis=1)
as_5 = np.nanmean(salt_5,axis=1)
at_1 = np.nanmean(temp_1,axis=1)
at_2 = np.nanmean(temp_2,axis=1)
at_3 = np.nanmean(temp_3,axis=1)
at_4 = np.nanmean(temp_4,axis=1)
at_5 = np.nanmean(temp_5,axis=1)

In [136]:
myarr = np.array([np.nan,1,3])
np.nanmean(myarr)

2.0

In [146]:
# plot floats ending as AASW colored by time
figure = plt.figure(figsize=[25, 25])
ax = plt.subplot(1,1,1)
plt.title('AABW')
plt.rcParams['axes.labelsize']='x-large'
plt.scatter(as_1,at_1,s=0.1,marker='o')
plt.scatter(as_2,at_2,s=0.1,marker='o')
plt.scatter(as_3,at_3,s=0.1,marker='o')
plt.scatter(as_4,at_4,s=0.1,marker='o')
plt.scatter(as_5,at_5,s=0.1,marker='o')
plt.plot(si,Freez_temp,'-',color='dimgray',label='Seawater Freezing',alpha=0.5,zorder=1)
#plt.plot(si,Tgade,'-',color='steelblue',label='Gade Line',alpha=0.5,zorder=2)
plt.ylim([-2.4,0.3])
plt.xlim([33.8,35.1])
plt.xlabel("Salinity",fontsize=15)
plt.ylabel("Temperature",fontsize=15)
plt.legend(loc="best",fontsize=30)
cs = ax.contour(si, ti, dens, colors="silver", zorder=1,alpha=0.5)
#ax.contour(si,ti,Freez_temp, colors="grey", zorder=1,alpha=0.5)
cl=plt.clabel(cs,fontsize=10,inline=False,fmt="%.1f")

plt.savefig("averagepts_db.png", dpi=300)
plt.close()

### Write the indexes to a text file

In [40]:
#aasw_idx, mcdw_idx, dsw_idx, isw_idx
f = open("aabw_idxs.txt", "w")

content = str(aabw_idx)
f.write(content)

f.close()

### Residence time at different depths

In [24]:
import math

In [33]:
aabw_cmap*=-1

In [34]:
# we will chunk the depths into
a = (aabw_cmap.flatten())
maxa = (max((a[~np.isnan(a)])))
mina = (min((a[~np.isnan(a)])))

# divide depths into containers of 500
mylist = np.r_[math.floor(mina):math.ceil(maxa):500]

In [35]:
mylist

array([   0,  500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500])

In [36]:
# digitize the numbers in a into the map
depth_bins = np.digitize(aabw_cmap,mylist)
depth_bins

array([[10, 10, 10, ..., 10, 10, 10],
       [10, 10, 10, ..., 10, 10, 10],
       [10, 10, 10, ..., 10, 10, 10],
       ...,
       [ 6,  5,  9, ...,  6,  9,  7],
       [ 6,  5,  9, ...,  6,  9,  7],
       [ 6,  5,  9, ...,  6,  9,  7]])

In [95]:
# divide the normal grid by 10
xbins = np.r_[0:5000:1]
xbins = np.arange(0,5000,500)

# index of bins the particles ending in
x_new = np.digitize(aabw_cmap,xbins)

In [97]:
x_new

array([[10, 10, 10, ..., 10, 10, 10],
       [10, 10, 10, ..., 10, 10, 10],
       [10, 10, 10, ..., 10, 10, 10],
       ...,
       [ 6,  5,  9, ...,  6,  9,  7],
       [ 6,  5,  9, ...,  6,  9,  7],
       [ 6,  5,  9, ...,  6,  9,  7]])

In [37]:
mesh

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])